In [1]:
# write a loop and save data
# import 
import obspy
from obspy.clients.fdsn import Client
client = Client("USGS")
from obspy import UTCDateTime
import numpy as np
from obspy.geodetics import locations2degrees
from obspy.geodetics import gps2dist_azimuth

import scipy
import scipy.signal as scisignal
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

from PIL import Image
from IPython.display import Image
from scipy import signal
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger
from scipy.interpolate import interp1d

import os
# import geopy.distance
import glob
import obspy.signal.filter
import datetime

In [2]:
## choose stations within the circle (-157,55 as center, 3 as radius)
import pandas as pd
obs = pd.read_csv('Seismometer_station',sep ='   ',header=None)
# print(obs.to_string())
sta_name = []
loc_name = []
rx = -157; ry = 55; rc = 3;
for istation in obs.index:
    disr = np.sqrt(np.square(obs.loc[istation,3]-rx)+np.square(obs.loc[istation,2]-ry))
    if disr<rc:
        sta_name.append(obs.loc[istation,1])
        loc_name.append(obs.loc[istation,0])

/home/bhe/.conda/envs/bhepy37/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
## download data
## Extract East component
## Merge data
## Band pass filter
## get envelope
## Downsample
## Save data
Direc = '2018-**-**'
datetime0 = datetime.datetime.strptime(Direc,'%Y-%m-%d')
l1 = 2; l2 = 8;
for iday in range(***):
    datetime1 = datetime0 +datetime.timedelta(days = iday)
    Direc = datetime.datetime.strftime(datetime1,'%Y-%m-%d')
## print time
    print(Direc)
    for istation in range(len(sta_name)):
##--------------get data -----------------##
        sta = sta_name[istation]
        loc = loc_name[istation]

        filename = Direc + '/' + sta + '.' + loc + '.mseed'
        
        try:
            st = obspy.read(filename)
        except:
            print("No file name")
        else:
##--------------choose East component---------------##
            if len(st.select(channel='BHE'))>=1:
                st_filt = st.select(channel='BHE').copy()

            elif len(st.select(channel='HHE'))>=1:
                st_filt = st.select(channel='HHE').copy()

            else:
                st_filt = st.select(channel='HH1').copy()

            cha = 'HH1'
##---------------merge the data------------------------##
            try:
                st_filt.merge(fill_value=0)
            except:
                for ist in range(len(st_filt)):
                    st_filt[ist].stats.sampling_rate = round(st_filt[ist].stats.sampling_rate,0)
                st_filt.merge(fill_value=0)
            print(st_filt)
##--------------Band pass filter l1-l2 Hz--------------##            
            st_filt1 = st_filt.copy()
            st_filt1.filter('bandpass',freqmin=l1,freqmax=l2)
            try: 
                print(np.std(st_filt1[0].data))
            except:
                print("0 trace")
            else:
##--------------get envelop---------------------------##
                data_envelope = obspy.signal.filter.envelope(st_filt1[0].data)
                samprate = int(st_filt1[0].stats.sampling_rate)
                npts = st_filt1[0].stats.npts
                
                if (samprate*60*60*24-1) <= npts: 
                    
##---------------- Downsample data--------------------##
                    b,a = scisignal.butter(N=3,Wn=0.2,btype='low',fs=samprate)
                    st_filt2 = scisignal.lfilter(b,a,data_envelope)
                    t_filt2 = np.arange(0, 24*60, 24*60/npts)
                    f_interp = interp1d(t_filt2,st_filt2,kind='nearest')
                    t_interp = np.arange(0, 24*60, 1/60)
                    st_interp = f_interp(t_interp)

##-------------------save data------------------------##
                    row1 = 3*60
                    col1 = len(np.arange(0,24*60-1.5,1.5))
                    npts2 = 3600*24
                    data = np.zeros((row1,col1))
                    for im in np.arange(0,24*60-1.5,1.5):
                        if np.std(st_filt[0].data[int(im*npts/24/60):int((im+3)*npts/24/60)]) == 0.0:
                            print('Seismic: 0')
                            data[:,int(im/1.5)] = 0*st_interp[int(im*npts2/24/60):int((im+3)*npts2/24/60)]
                        else:
                            data[:,int(im/1.5)] = st_interp[int(im*npts2/24/60):int((im+3)*npts2/24/60)]
#                     print(data.shape)
                    with open(Direc + '/' + sta +'_'+ cha +'_180s_lf_'+str(l1)+'-'+str(l2)+'.npy','wb') as f:
                        np.save(f,data)



2018-05-10
1 Trace(s) in Stream:
AK.CHI..BHE | 2018-05-10T00:00:00.008400Z - 2018-05-10T23:59:59.988400Z | 50.0 Hz, 4320000 samples
143.924508392
No file name
1 Trace(s) in Stream:
AT.CHGN..BHE | 2018-05-10T00:00:00.000000Z - 2018-05-11T00:00:00.000000Z | 50.0 Hz, 4320001 samples
376.083026607
1 Trace(s) in Stream:
TA.R16K..BHE | 2018-05-10T00:00:00.000000Z - 2018-05-11T00:00:00.000000Z | 40.0 Hz, 3456001 samples
99.5250937423
No file name
No file name
1 Trace(s) in Stream:
TA.S14K..BHE | 2018-05-10T00:00:00.000000Z - 2018-05-11T00:00:00.000000Z | 40.0 Hz, 3456001 samples
196.965898503
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No f

/home/bhe/.conda/envs/bhepy37/lib/python3.7/site-packages/obspy/core/stream.py:2958: UserWarning: Incompatible traces (sampling_rate, dtype, ...) with same id detected. Doing nothing.
  warnings.warn(msg)


1 Trace(s) in Stream:
AK.CHI..BHE | 2018-05-12T00:46:43.488400Z - 2018-05-12T23:59:59.988400Z | 50.0 Hz, 4179826 samples
60.5665360212
1 Trace(s) in Stream:
AK.CHN..BHE | 2018-05-12T00:00:00.008400Z - 2018-05-12T23:59:59.988400Z | 50.0 Hz, 4320000 samples
21.7800016347
1 Trace(s) in Stream:
AT.CHGN..BHE | 2018-05-12T00:00:00.000000Z - 2018-05-13T00:00:00.000000Z | 50.0 Hz, 4320001 samples
49.316384899
1 Trace(s) in Stream:
TA.R16K..BHE | 2018-05-12T00:00:00.000000Z - 2018-05-13T00:00:00.000000Z | 40.0 Hz, 3456001 samples
79.4249141802
No file name
No file name
1 Trace(s) in Stream:
TA.S14K..BHE | 2018-05-12T00:00:00.000000Z - 2018-05-13T00:00:00.000000Z | 40.0 Hz, 3456001 samples
62.5558629472
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
No file name
1 Trace(s) in Stream:
XO.LT05..HH1 | 2018-05-12T09:40:21.461200Z - 2018-05-1

In [4]:
# st = obspy.read('2018-11-25/CHI.AK.mseed')
# st_filt = st.select(channel='BHE').copy()
# print(st_filt)
# st_filt.merge(fill_value=0)
# st_filt1 = st_filt.copy()
# st_filt1.filter('bandpass',freqmin=l1,freqmax=l2)
# try: 
#     print(np.std(st_filt1[0].data))
# except:
#     print("0 trace")
# else:
#     ##--------------get envelop-----------------------##
#     data_envelope = obspy.signal.filter.envelope(st_filt1[0].data)
#     samprate = int(st_filt1[0].stats.sampling_rate)
#     npts = st_filt1[0].stats.npts

#     ##---------------- Down sample data------------------------------##
#     b,a = scisignal.butter(N=3,Wn=0.2,btype='low',fs=samprate)
#     st_filt2 = scisignal.lfilter(b,a,data_envelope)
#     t_filt2 = np.arange(0, 24*60, 24*60/npts)
#     f_interp = interp1d(t_filt2,st_filt2,kind='nearest')

In [5]:
# import os
# Direc = '2018-10-01'
# datetime0 = datetime.datetime.strptime(Direc,'%Y-%m-%d')
# ## set up a day
# for iday in range(60):
#     datetime1 = datetime0 +datetime.timedelta(days = iday)
#     Direc = datetime.datetime.strftime(datetime1,'%Y-%m-%d')
#     sta_name = glob.glob(Direc + '/FALS*.npy')
#     for ista in range(len(sta_name)):
#         print(sta_name[ista])
#         os.remove(sta_name[ista])